In [0]:
# configs = {"fs.azure.account.key.storage001de.blob.core.windows.net": "MA38y0ZA2Ij1L3QXl/F/Y8EGXR1VjTGb896PNbUmPT99uhVLoi0qCEiBEBagBQ4yscqwq9Zxb0vX+AStlr0Z3Q=="}

# # create mount to silver
# dbutils.fs.mount(
# 	source = "wasbs://silver@storage001de.blob.core.windows.net/",
# 	mount_point = "/mnt/datalake/silver",
# 	extra_configs = configs
# )

In [0]:
%fs ls /mnt/datalake/silver

path,name,size,modificationTime
dbfs:/mnt/datalake/silver/cleaned_cast/,cleaned_cast/,0,0
dbfs:/mnt/datalake/silver/cleaned_company/,cleaned_company/,0,0
dbfs:/mnt/datalake/silver/cleaned_director/,cleaned_director/,0,0
dbfs:/mnt/datalake/silver/cleaned_genre/,cleaned_genre/,0,0
dbfs:/mnt/datalake/silver/cleaned_keyword/,cleaned_keyword/,0,0
dbfs:/mnt/datalake/silver/cleaned_movie/,cleaned_movie/,0,0
dbfs:/mnt/datalake/silver/cleaned_movie_company/,cleaned_movie_company/,0,0
dbfs:/mnt/datalake/silver/cleaned_movies_metadata/,cleaned_movies_metadata/,0,0


In [0]:
df = spark.read.table("silver.movie")
df.show(5)

+-----+---------+--------------------+----------+--------------------+------+---------+-------+--------------------+--------+--------------------+--------------------+----------+------+-------+--------------------+--------------------+--------------------+
|   id|   imdbId|               title|   release|                plot|rating|voteCount|tagline|              genres|language|           companies|           countries|popularity|budget|revenue|                cast|                crew|            keywords|
+-----+---------+--------------------+----------+--------------------+------+---------+-------+--------------------+--------+--------------------+--------------------+----------+------+-------+--------------------+--------------------+--------------------+
|22091|tt0000008|Edison Kinetoscop...|1894-01-09|A man (Thomas Edi...|   5.4|     2147|   NULL|[{'id': 99, 'name...|      en|[{'name': 'Edison...|[{'iso_3166_1': '...|  0.312246|     0|      0|[{'cast_id': 2, '...|[{'credit_id': 

In [0]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
# Tạo DataFrame từ năm 1900 đến 2023
years = range(1900, 2024)  # 1900 - 2023
months = range(1, 13)  # 1 - 12

data = []

# Tạo dữ liệu cho DataFrame với các giá trị id, year, month
for year in years:
    for month in months:
        data.append((year * 100 + month, year, month))

# Tạo DataFrame từ dữ liệu được tạo ra
columns = ["id", "year", "month"]
date_df = spark.createDataFrame(data, columns)

# Gán id tăng dần
window = Window.orderBy("year", "month")
date_df = date_df.withColumn("id", row_number().over(window))
# Hiển thị một số dòng đầu của bảng
date_df.show()

+---+----+-----+
| id|year|month|
+---+----+-----+
|  1|1900|    1|
|  2|1900|    2|
|  3|1900|    3|
|  4|1900|    4|
|  5|1900|    5|
|  6|1900|    6|
|  7|1900|    7|
|  8|1900|    8|
|  9|1900|    9|
| 10|1900|   10|
| 11|1900|   11|
| 12|1900|   12|
| 13|1901|    1|
| 14|1901|    2|
| 15|1901|    3|
| 16|1901|    4|
| 17|1901|    5|
| 18|1901|    6|
| 19|1901|    7|
| 20|1901|    8|
+---+----+-----+
only showing top 20 rows



#### Create Table


In [0]:
from delta.tables import *
from pyspark.sql.functions import col, substring

# #check if the silver location contain the delta table
# if (DeltaTable.isDeltaTable(spark, movie_silverLocation)):
#     DeltaTable.forPath(spark, movie_silverLocation).alias("target").merge(
#         source = cleaned_movie.alias("src"),
#         condition = "target.id = src.id"
#     ).WhenNoMatchedInsertAll().execute()
# else:

# Define the silver location

# Define the silver location
date_silverLocation = "/mnt/datalake/silver/cleaned_date"

# Table name
date_silverTable = "silver.date"

date_df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(date_silverLocation)

In [0]:
# create the schema and table, if required
# spark.databricks.delta.formatCheck.enabled false

spark.sql(f"CREATE EXTERNAL TABLE IF NOT EXISTS {date_silverTable} USING delta LOCATION '{date_silverLocation}'")


DataFrame[]

In [0]:
# Maintenance for Delta table
date_DataDelta = DeltaTable.forName(spark, date_silverTable)

if date_DataDelta.history(30).filter("operation = 'VACUUM START'").count() == 0:
	date_DataDelta.optimize()
	date_DataDelta.vacuum()


In [0]:
%sql
DESCRIBE HISTORY silver.date;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2023-12-17T10:15:39Z,1311347395691033,totnvt02@gmail.com,VACUUM END,Map(status -> COMPLETED),null,List(3918195481499330),1122-230829-ksqo7xkx,1,SnapshotIsolation,true,"Map(numDeletedFiles -> 0, numVacuumedDirectories -> 1)",null,Databricks-Runtime/13.3.x-scala2.12
1,2023-12-17T10:15:35Z,1311347395691033,totnvt02@gmail.com,VACUUM START,"Map(retentionCheckEnabled -> true, defaultRetentionMillis -> 604800000)",null,List(3918195481499330),1122-230829-ksqo7xkx,0,SnapshotIsolation,true,"Map(numFilesToDelete -> 0, sizeOfDataToDelete -> 0)",null,Databricks-Runtime/13.3.x-scala2.12
0,2023-12-17T10:15:19Z,1311347395691033,totnvt02@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,List(3918195481499330),1122-230829-ksqo7xkx,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1488, numOutputBytes -> 7968)",null,Databricks-Runtime/13.3.x-scala2.12


In [0]:
%sql
select * from silver.date 

id,year,month
1,1900,1
2,1900,2
3,1900,3
4,1900,4
5,1900,5
6,1900,6
7,1900,7
8,1900,8
9,1900,9
10,1900,10
